In [1]:
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from utils import file_to_coupling_map, directory_to_circuits, build_pass_manager
import random
import time
import csv
import os
import numpy as np



In [2]:
def run_experiment(cm_name, qc_name, layout_trials_list=[1, 10, 100, 1000], heuristic="lookahead", num_pass_trials=5, seed=42, output_csv="experiments/experiment_new.csv"):
    """
    Runs a transpilation experiment using SABRE passes on a set of quantum circuits.

    Parameters:
    - cm_name (str): Name of the coupling map file (without extension) located in 'coupling_maps/' directory.
    - qc_name (str): Name of the directory containing quantum circuits to run the experiment on.
    - layout_trials_list (list): List of layout (and swap) trials to run for each circuit. Default is [1, 10, 100, 1000].
    - heuristic (str): Heuristic to use in SabreSwap. Default is "lookahead".
    - num_pass_trials (int): Number of different seeds for running the whole pass. Default is 1.
    - seed (int): Random seed for reproducibility. Default is 42.
    - output_csv (str): Filename for the CSV file where results will be saved. Default is "experiment_results.csv".
    
    Returns:
    - None (Results are saved to a CSV file)
    """
    
    # Set random seed for reproducibility
    random.seed(seed)

    # Load the coupling map and the quantum circuits
    cm = CouplingMap(file_to_coupling_map(f'coupling_maps/{cm_name}.txt'))
    qc_list = directory_to_circuits(f'{qc_name}')

    # Prepare the CSV file
    with open(output_csv, mode='w', newline='') as csvfile:
        fieldnames = ["num_qubits", "layout_trials", "heuristic", "avg_runtime", "avg_swap_count", "avg_depth", "num_pass_trials"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Run trials for each circuit
        for qc in qc_list:
            num_qubits = qc.num_qubits
            print(f"Running experiment for {num_qubits} qubits")

            for layout_trials in layout_trials_list:
                seed_list = random.sample(range(0, 100000), num_pass_trials)

                total_runtime = 0
                total_swap_count = 0
                total_depth = 0

                for trial in range(num_pass_trials):
                    # Create the pass manager with the specified layout trials
                    rp = SabreSwap(coupling_map=cm, heuristic=heuristic, seed=seed_list[trial], trials=layout_trials)
                    lp = SabreLayout(coupling_map=cm, routing_pass=rp, seed=seed_list[trial])
                    pm = build_pass_manager(cm, lp, rp)

                    # Run the transpilation pass
                    time_start = time.time()
                    qc_tr = pm.run(qc)
                    time_end = time.time()

                    # Collect the metrics for this trial
                    runtime = time_end - time_start
                    swap_count = qc_tr.count_ops().get('swap', 0)
                    depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2)  # 2-qubit gates

                    total_runtime += runtime
                    total_swap_count += swap_count
                    total_depth += depth

                    print(f"Layout trials: {layout_trials}, Seed: {seed_list[trial]} - Trial {trial+1}/{num_pass_trials} - Runtime: {runtime:.10f} sec, Swap count: {swap_count}, Depth: {depth}")

                # Calculate averages
                avg_runtime = total_runtime / num_pass_trials
                avg_swap_count = total_swap_count / num_pass_trials
                avg_depth = total_depth / num_pass_trials

                # Store averaged results in CSV
                writer.writerow({
                    "num_qubits": num_qubits,
                    "layout_trials": layout_trials,
                    "heuristic": heuristic,
                    "avg_runtime": avg_runtime,
                    "avg_swap_count": avg_swap_count,
                    "avg_depth": avg_depth,
                    "num_pass_trials": num_pass_trials
                })

    print(f"Results saved to {output_csv}")

In [3]:
# Example usage:

heuristic = "decay"

# Generate log scale values
log_values = np.linspace(0, 3, 142)
linear_values = np.round(10 ** log_values).astype(int)
unique_values = np.unique(np.clip(linear_values, 0, 1000))
layout_trials_list = unique_values.tolist()

run_experiment(cm_name='heavy_hex_127', qc_name='ghz', heuristic=heuristic, layout_trials_list=layout_trials_list, num_pass_trials=5, output_csv=f"experiments/ghz_{heuristic}.csv")

Running experiment for 10 qubits
Layout trials: 1, Seed: 83810 - Trial 1/5 - Runtime: 0.1572051048 sec, Swap count: 29, Depth: 38
Layout trials: 1, Seed: 14592 - Trial 2/5 - Runtime: 0.0441229343 sec, Swap count: 25, Depth: 33
Layout trials: 1, Seed: 3278 - Trial 3/5 - Runtime: 0.0950047970 sec, Swap count: 27, Depth: 36
Layout trials: 1, Seed: 97196 - Trial 4/5 - Runtime: 0.0742812157 sec, Swap count: 22, Depth: 30
Layout trials: 1, Seed: 36048 - Trial 5/5 - Runtime: 0.0929300785 sec, Swap count: 11, Depth: 19
Layout trials: 2, Seed: 32098 - Trial 1/5 - Runtime: 0.1510996819 sec, Swap count: 12, Depth: 21
Layout trials: 2, Seed: 29256 - Trial 2/5 - Runtime: 0.0646028519 sec, Swap count: 15, Depth: 24
Layout trials: 2, Seed: 18289 - Trial 3/5 - Runtime: 0.1710159779 sec, Swap count: 35, Depth: 44
Layout trials: 2, Seed: 96530 - Trial 4/5 - Runtime: 0.1567018032 sec, Swap count: 7, Depth: 16
Layout trials: 2, Seed: 13434 - Trial 5/5 - Runtime: 0.1419188976 sec, Swap count: 23, Depth: 32